In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
from generate_conanfiles import *

repo_dir = os.path.abspath('')
src_dir = os.path.join(repo_dir, "src")
package_xmls = get_package_xml_files(src_dir)
make_abs = lambda x: os.path.abspath(os.path.join(src_dir, x))
package_dirs = list(map(lambda x: Path(make_abs(x)).parent, package_xmls))


In [20]:
import pandas as pd
from generate_conanfiles import has_cmakelists, has_setup_py
import xml.etree.ElementTree as ET
import copy


def create_package_meta(dirname):
    return {
        "dir" : dirname,
        "cmake" : has_cmakelists(dirname),
        "python" : has_setup_py(dirname),
    }

def get_dependencies(package):
    """
    Gather data from export section of package.xml as defined by the spec https://ros.org/reps/rep-0149.html#dependency-tags
    Dependency tags
        <build_depend> (multiple)
            Attributes
        <build_export_depend> (multiple)
            Attributes
        <buildtool_depend> (multiple)
            Attributes
        <buildtool_export_depend> (multiple)
            Attributes
        <exec_depend> (multiple)
            Attributes
        <depend> (multiple)
            Attributes
        <doc_depend> (multiple)
            Attributes
        <test_depend> (multiple)
            Attributes
        <conflict> (multiple)
            Attributes
        <replace> (multiple)
            Attributes
    """
    tree = ET.parse(package)
    root = tree.getroot()

    dependency_tags = {
        "build_depend" : False,
        "build_export_depend" : False,
        "buildtool_depend": False,
        "exec_depend" : False,
        "depend" : False,
        "doc_depend": False,
        "test_depend" : False,
        "conflict": False, # probably inactionble data - very very rarely used
        "replace": False,  # probably inactionble data - very very rarely used
    }

    dependency_tag_names = dependency_tags.keys()

    dependencies = {child.text: copy.deepcopy(dependency_tags) for child in root if child.tag in dependency_tag_names}

    for child in root:
        if child.tag in dependency_tag_names:
            dependencies[child.text][child.tag] = True

    return dependencies

def get_build_types(package):
    """ 
    Gather build types from export section of package.xml as defined by the spec https://ros.org/reps/rep-0149.html#export
    <export>
        <architecture_independent/>
        <build_type> (multiple)
            Attributes
        <deprecated>
        <message_generator>
        <metapackage/>  - very very rarely used
    """

    tree = ET.parse(package)
    root = tree.getroot()

    build_types = []

    export_element = root.find('export')
    if export_element is not None:
        build_types = [child.text for child in export_element if child.tag == "build_type"]
    
    return build_types

def package_name(dirname):
    return os.path.split(dirname)[-1]

pkg_info = { package_name(dirname) : create_package_meta(dirname) for dirname in package_dirs}

pkg = os.path.join(package_dirs[-1], 'package.xml')
print(pkg)
with open(pkg, 'r') as stream:
    print(stream.read())
deps = get_dependencies(pkg)
build_types = get_build_types(pkg)

# df = pd.DataFrame(pkg_info).transpose()
df = pd.DataFrame(deps)
df.to_excel("package_info.xlsx", sheet_name="deps")
print(build_types)


/home/tbitz/repos/ros2conan/src/ros-tooling/libstatistics_collector/package.xml
<?xml version="1.0"?>
<?xml-model href="http://download.ros.org/schema/package_format3.xsd" schematypens="http://www.w3.org/2001/XMLSchema"?>
<package format="3">
  <name>libstatistics_collector</name>
  <version>1.3.1</version>
  <description>Lightweight aggregation utilities to collect statistics and measure message metrics.</description>
  <maintainer email="ros-tooling@googlegroups.com">ROS Tooling Working Group</maintainer>
  <license>Apache License 2.0</license>

  <buildtool_depend>ament_cmake</buildtool_depend>
  <buildtool_depend>ament_cmake_ros</buildtool_depend>

  <build_depend>rosidl_default_generators</build_depend>
  <build_depend>std_msgs</build_depend>

  <depend>builtin_interfaces</depend>
  <depend>rcl</depend>
  <depend>rcpputils</depend>
  <depend>statistics_msgs</depend>

  <exec_depend>rosidl_default_runtime</exec_depend>
  <exec_depend>std_msgs</exec_depend>

  <test_depend>ament_cma

In [7]:
build_system_unsure = [] 
both_cmake_and_python= [];
for pkg, meta in pkg_info.items():
    if (not meta['cmake'] and not meta['python']):
        build_system_unsure.append(pkg)
    if (meta['cmake'] and meta['python']):
        both_cmake_and_python.append(pkg)

print(f"Both cmake and python: {both_cmake_and_python}")
print(f"Neither cmake or python: {build_system_unsure}")

for pkg in both_cmake_and_python:
    pkg_dir = pkg_info[pkg]['dir']
    print(f"[{pkg}]\n{os.listdir(pkg_dir)}\n\n")

NameError: name 'dir_meta' is not defined